## Initial notebook for project 

In [1]:
# Imports and set up
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset

from torcheeg.io.eeg_signal import EEGSignalIO

## Path to dir with data (remember the last '/')
path = "../data/"

## Establish connection to datafile
IO = EEGSignalIO(io_path=str(path), io_mode='lmdb')
## Read metadata dataframeimports
metadata = pd.read_csv(path + 'sample_metadata.tsv', sep='\t')

In [2]:
# Verifying connextion to data
idxs = np.arange(len(metadata))

eeg = torch.FloatTensor(np.array([IO.read_eeg(str(i)) for i in idxs]))
print(f"nsamples: {eeg.shape[0]}  -  nchannels: {eeg.shape[1]}  -  t: {eeg.shape[2]}")

nsamples: 5184  -  nchannels: 22  -  t: 800


Imports we might need cuz eeg ref code.

In [3]:
#set up of matrixies 
#number of samples, channels, and timesteps
nsamples, nchannels, t = eeg.shape  

labels = torch.tensor(metadata["value"].values, dtype=torch.long)

train_dataset = TensorDataset(eeg, labels)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

adj_matrix = torch.eye(eeg.shape[1])  #place holder change later FIX

In [4]:
class GraphConv(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConv, self).__init__()
        self.weight = nn.Parameter(torch.randn(in_features, out_features))
        self.bias = nn.Parameter(torch.zeros(out_features))

    def forward(self, x, adj):
        x = torch.matmul(adj, x)  
        x = torch.matmul(x, self.weight) + self.bias  
        return torch.relu(x)

import torch.nn.functional as F

class EEG_GNN(nn.Module):
    def __init__(self, in_features, hidden_dim, nclasses, nchannels):
        super(EEG_GNN, self).__init__()
        self.conv1 = GraphConv(in_features, hidden_dim)
        self.batch_norm1 = nn.BatchNorm1d(nchannels) 
        self.conv2 = GraphConv(hidden_dim, hidden_dim)
        self.batch_norm2 = nn.BatchNorm1d(nchannels) 
        self.conv3 = GraphConv(hidden_dim, nclasses)  

    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = self.batch_norm1(x)  
        x = F.relu(x)
        x = self.conv2(x, adj)
        x = self.batch_norm2(x)  
        x = F.relu(x)
        x = self.conv3(x, adj)  
        x = x.mean(dim=1)  
        return x

In [5]:
class TrainGNN():
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def train_model(self, model, train_loader, adjacency_matrix, learning_rate=0.0005, epochs=500):
        model = model.to(self.device)
        adjacency_matrix = adjacency_matrix.to(self.device)  # Move adjacency to GPU if available

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        highest_train_accuracy = 0.0

        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            for inputs, labels in train_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                optimizer.zero_grad()
                outputs = model(inputs, adjacency_matrix)  # Forward pass
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            epoch_loss = running_loss / len(train_loader.dataset)
            epoch_accuracy = correct / total
    
            if epoch_accuracy > highest_train_accuracy:
                highest_train_accuracy = epoch_accuracy

            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {(epoch_accuracy*100):.2f}%")

        print(f"Highest Trained Accuracy {(highest_train_accuracy*100):.2f}")
        torch.save(model.state_dict(), 'eeg_gnn.pth')

        return model


In [ ]:
nclasses = labels.max().item() + 1 
model = EEG_GNN(in_features=t, hidden_dim=32, nclasses=nclasses, nchannels=nchannels)

trainer = TrainGNN()
trained_model = trainer.train_model(model, train_loader, adj_matrix,epochs=200)

Epoch 1/200, Loss: 3.0114, Accuracy: 12.38%
Epoch 2/200, Loss: 2.2961, Accuracy: 17.23%
Epoch 3/200, Loss: 1.9820, Accuracy: 19.95%
Epoch 4/200, Loss: 1.8298, Accuracy: 21.12%
Epoch 5/200, Loss: 1.7435, Accuracy: 22.45%
Epoch 6/200, Loss: 1.6912, Accuracy: 23.15%
Epoch 7/200, Loss: 1.6566, Accuracy: 23.92%
Epoch 8/200, Loss: 1.6347, Accuracy: 24.52%
Epoch 9/200, Loss: 1.6189, Accuracy: 24.92%
Epoch 10/200, Loss: 1.6058, Accuracy: 25.56%
Epoch 11/200, Loss: 1.5973, Accuracy: 26.08%
Epoch 12/200, Loss: 1.5891, Accuracy: 26.49%
Epoch 13/200, Loss: 1.5811, Accuracy: 26.70%
Epoch 14/200, Loss: 1.5740, Accuracy: 27.49%
Epoch 15/200, Loss: 1.5681, Accuracy: 27.91%
Epoch 16/200, Loss: 1.5624, Accuracy: 28.34%
Epoch 17/200, Loss: 1.5563, Accuracy: 28.97%
Epoch 18/200, Loss: 1.5502, Accuracy: 29.48%
Epoch 19/200, Loss: 1.5442, Accuracy: 29.57%
Epoch 20/200, Loss: 1.5375, Accuracy: 30.32%
Epoch 21/200, Loss: 1.5315, Accuracy: 30.57%
Epoch 22/200, Loss: 1.5248, Accuracy: 31.21%
Epoch 23/200, Loss: